> Se importan las librerías a utilizar

In [0]:
#spark =SparkSession.builder.appName("mi_app").getOrCreate()
import numpy as np

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import col

from pyspark.sql.types import StringType,DoubleType
import pyspark.sql.functions as F
from pyspark.sql.types import *


from decimal import *
sc=spark.sparkContext

from pyspark.sql.functions import lit
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col
import csv
import numpy as np
import pandas as pd
import gzip
from datetime import date
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score
from numpy import arange
from matplotlib import pyplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql.functions import col

from pyspark.sql.types import StringType,DoubleType
from pyspark.sql.types import (StringType, BooleanType, IntegerType, FloatType, DateType)
  
from pyspark.sql.functions import col,isnan, when, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import Window


#Librerias de Spark que se utilizan en la generacion del modelo de regresion lineal 
from pyspark.sql.functions import corr


from pyspark.ml.regression import LinearRegression



from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator


In [0]:
%sh
pip install mlflow
/databricks/python_shell/scripts/PythonShell.py configure

Requirement already satisfied: mlflow in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (1.27.0)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Requirement already satisfied: Flask in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (2.1.3)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.6.2)
Requirement already satisfied: alembic in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (1.8.1)
Requirement already satisfied: pandas in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (1.4.3)
Requirement already satisfied: sqlparse>=0.3.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (0.4.2)
Requirement already satisfied: protobuf>=3.12.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.2)
Requirement already satisfied: gitpython>=2.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (3.1.27)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Requirement already satisfied: databricks-cli>=0.8.7 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (0.17.0)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.9)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.25.1)
Requirement already satisfied: cloudpickle in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (2.1.0)
Requirement already satisfied: click>=7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (8.1.3)
Requirement already satisfied: sqlalchemy>=1.4.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (1.4.39)
Requirement already satisfied: gunicorn in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (20.1.0)
Requirement already satisfied: docker>=4.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (5.0.3)
Requirement already satisfied: querystring-parser in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: pyyaml>=5.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (6.0)
Requirement already satisfied: importlib-metadata!=4.7.0,>=3.7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (4.12.0)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: prometheus-flask-exporter in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from mlflow) (0.20.2)
Requirement already satisfied: oauthlib>=3.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (3.2.0)
Requirement already satisfied: tabulate>=0.7.7 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.

In [0]:
import mlflow
import mlflow.spark

> Se importan las 3 bases que fueron el resultado del proceso de preparación de los datos:

> Se hace un filtrado para determinadas columnas

> Se categorizan valores numéricos

In [0]:



path_to_data="dbfs:/dbfs/gold"
#path_to_data="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabanafinal/"
#raw_datax = spark.read.options(inferSchema='True',header='True').format("delta").load(path_to_data)
raw_datax = spark.read.options(inferSchema='True',header='True').format("delta").load(path_to_data)
#('/dbfs/gold/icbf-gai/BaseModeloProbabilisticoSabana')



In [0]:

def tiposendf(raw_datax):# funcion que da los tipos en el df en una lista 
    tipos=raw_datax.dtypes
    ele=[]
    for i in range(len(tipos)):
        ele.append(tipos[i][1])
    uniq = []
    for x in ele:
        if x not in tipo:
            uniq.append(x)
            tipo.add(x)
    return(uniq)
  
def filtrotype(df, tipo):# funcion que da las columnas de cierto tipo en especifico 
    my_list = []
    for i in df.dtypes:
        if i[1].startswith(tipo):
            my_list.append(i)
    return my_list

def filtro(df, ini,fin): # filtro al cual entra como parametro un df y 2 palabras claves de inicio y fin del nombre de las columnas y devuelve un df que no tuene dichas columnas 
    lista=df.columns
    my_list = []
    for i in lista:
        if i.startswith(ini) and i.endswith(fin):
             my_list.append(i)
        
    df = df.drop(*my_list)
    return df

def filtro1(df, key,idi): # funcion  a la cual entra un df y dos palabras clves entre comillas y devuelve la lista de las columnas que tienen esas palabras 
    lista=df.columns
    arr = np.array(lista)
    my_list = []
    for i in arr:
        #for j in len(lista[i]):
        if key in i:
            my_list.append(i)
                
        if idi in i:
            my_list.append(i)
    return my_list

 ## fun repetidas 

def filtro2(df, key): # filtro al cual entra como parametro un df y  una palabra clave en comillas y devuelve una lista con las columnas del df que contienen esa palabra clave 
    lista=df.columns
    arr = np.array(lista)
    my_list = []
    for i in arr:
        #for j in len(lista[i]):
        if key in i:
            my_list.append(i)
                
    return my_list

def filtrofin(df, fin): # Filtro al cual entra un df y una palabra clave y  devuelve una lista con el nombre de todas las columnas que terminen con dichas ´palabras clave 
    lista=df.columns
    my_list = []
    for i in lista:
        if  i.endswith(fin):
             my_list.append(i)

    return my_list

In [0]:
len(raw_datax.columns)

Out[11]: 718

# limpieza 

## se utilizan los filtros para eliminar columnas repetidas o que no aportan informacion relevante, como identificacion por nombres, direcciones, fechas o columnas que ya estan modeladas por otras columnas en el df 

se eliminan 210 columnas con dichos filtros

# funciones para limpiar y crear las sabanas de datos

In [0]:
def nulls(df):# funcion que da el procentaje de nulls por columna de un df de entrada
    cols=df.columns
    rows=df.count()
    df=df.withColumn('numNulls1',sum(df[col].isNull().cast('int') for col in cols))
    
    #for col in cols:
        #if df.numNulls1
    Dict_Null = {col:df.filter(df[col].isNull()).count()/rows for col in cols}# crea un diccionario con
    #el nombre de la columna y la cantidad de nulls en    dicha columna
    val=Dict_Null.values() 
    key=Dict_Null.keys() 
    key1=list(key)#lista con nombre de la columna
    filt=[]
    val1=list(val)
    return(Dict_Null)
    # crea un diccionariocon el nombre de la columna y la cantidad de nulls en dicha columna
    

In [0]:
# la funcion filtro 5 elimina las columnas asociadas a las tomas no deseadas se debe especificar el # de tomas que se quieren tener en cuenta, 


def filtro5(df,Notomas):
    df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null

    p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
    p1=p.drop(*filtro2(df, 'IdBeneficiarioT')) #dataframe sin las columnas  IdBeneficiarioT
    listaEPT=p1.columns # lista con las columnas EstadoPesoTalla+EPT+ Idbeneficiario
    
    f=len(listaEPT) # cantidad de tomas mas el id del ben 
    
    #h=(f-Notomas+1)# columnas EstadoPesoTalla totales  -Notomas deseado- columna Idben=columnas EstadoPesoTalla nodeseadas
    #p1 =p1.filter(p1.numNulls== h) # filtra las columnas segun el no de tomas deseado
    
    listaEPTf=p1.columns # cantidad de tomas mas el id del ben sin columna num nulls

    lista3=listaEPTf[Notomas:]# lista con las columnas no deseadas 
    #deberia crear una columna que diga cuantas tomas tienen originalmente 
    p1=p1.drop(*lista3) # dataframe con las columnas de las EPT deseadas
    colfin=p1.columns

    lista4=p1.select('IdBeneficiario').rdd.map(lambda x : x[0]).collect()# hace una lista con los usuarios que cumplen la condicion de tener las tomas 
    #parte2

    df1=df.filter(df.IdBeneficiario.isin(lista4)) # elimina las filas de los usuarios que no cumplen en el df de entrada
    #lista5=df1.columns #columnas finale 
    #df1=df1.withColumn('numNulls1', sum(df1[col].isNull().cast('int') for col in lista5)) # crea una columna contando los nulls por fila 
    df1=df1.drop(*lista3)
    r=df1.count()
    c=df1.columns
    notomas=(range(1,len(c),1))
    lista6=[]
    for c in notomas: # crea una lista con el nombre de las columnas de las tomas que no se consideran 
        name='T'
        if c in lista3:
        #lista6.append(filtrofin(df2, "%s%s"%(name,c))
            lista6.extend(filtrofin(df2, "%s%s"%(name,c) ))
        else
    df2=df2.drop(*lista6) # elimina las columnas relacionadas con las tomas que no se tienen en cuenta

    nom=filtro1(df2, 'nom','Nom')
    idi=filtro2(df2, 'IdBeneficiarioT')
    ins =filtro2(df2, 'INS')
    Dir=filtro1(df2, 'Dir','dir')
    fecha=filtro1(df2, 'Fecha','fecha')
    flag=filtro1(df2, 'Flag','PresentaCarneVacunacion')
    NoTomaT=filtro1(df2, 'Flag','NoTomaT')
    TomaT=filtro2(df2, 'TomaT')
    df2=df2.drop(*nom)
    df2=df2.drop(*idi)
    df2=df2.drop(*Dir)
    df2=df2.drop(*fecha)
    df2=df2.drop(*flag)
    df2=df2.drop(*NoTomaT)
    df2=df2.drop(*TomaT)
    df2=df2.drop(*ins)

    listeli=['depmuni','depto','munic','zona','sector','seccion','manzana','comuna','barrio','vereda','NOM_DPTO','NOM_MPIO','NumeroDocumento',
    'SegundoApellido','PrimerApellido','NumeroDocumento','NoTomaT','ficha','teles','ape1','ape2','documen','ind_estudia','tip_origen_agua','ind_tiene_alcantarillado','tip_mat_paredes','tip_mat_pisos','ind_tiene_acueducto','num_cuartos_dormir','tip_sanitario','ide_ficha_origen','num_ficha', 'Jefe_UG',  'seg_nom_informante',
    'tip_cuidado_niños',
    'ide_firma_informante',
    'Email_contacto',
    'Cod_dpto',
    'Cod_mpio',
    'coord_x_manual_rec',
    'coord_y_manual_rec',
    'coord_x_auto_rec',
    'coord_y_auto_rec',
    'Dir_Chip',
    'ide_persona',
    'ind_escaner',
    'pri_apellido',
    'seg_apellido',
    'pri_nombre',
    'seg_nombre',
    'sexo_persona',
    'tip_documento',
    'num_documento',
    'fec_nacimiento',
    'edad_calculada',
    'fec_documento',
    'cod_mpio_documento',
    'cod_dpto_documento',
    'cod_pais_documento',
    'ind_discap_ver',
    'ind_discap_oir',
    'ind_discap_hablar',
    'ind_discap_moverse',
    'ind_discap_bañarse',
    'ind_discap_salir',
    'ind_discap_entender',
    'ind_discap_ninguna',
    'ind_fondo_pensiones',
    'tip_actividad_mes',
    'num_sem_buscando',
    'tip_empleado',
    'Ide_hogar',
    'ide_informante',
    'pri_nom_informante',
    'seg_nom_informante',
    'pri_ape_informante',
    'seg_nom_informante',
    'ide_firma_informante',
    'Email_contacto',
    'num_tel_contacto',
    'Grupo',
    'Nivel',
    'fec_actualizacion_cns',
    'tip_mat_paredes',
    'tip_mat_pisos']
    df2=df2.drop(*listeli)
    #ZonaUbicacionBeneficiario
    return df2

In [0]:

df=raw_datax.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in raw_datax.columns]) #llena las columnas con null
Notomas=17
p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
p1=p.drop(*filtro2(df, 'IdBeneficiarioT')) #dataframe sin las columnas  IdBeneficiarioT
listaEPT=p1.columns # lista con las columnas EstadoPesoTalla+EPT+ Idbeneficiario

f=len(listaEPT) # cantidad de tomas mas el id del ben 

#h=(f-Notomas+1)# columnas EstadoPesoTalla totales  -Notomas deseado- columna Idben=columnas EstadoPesoTalla nodeseadas
#p1 =p1.filter(p1.numNulls== h) # filtra las columnas segun el no de tomas deseado

listaEPTf=p1.columns # cantidad de tomas mas el id del ben sin columna num nulls

lista3=listaEPTf[Notomas:]# lista con las columnas no deseadas 
#deberia crear una columna que diga cuantas tomas tienen originalmente 
p1=p1.drop(*lista3) # dataframe con las columnas de las EPT deseadas
colfin=p1.columns

lista4=p1.select('IdBeneficiario').rdd.map(lambda x : x[0]).collect()# hace una lista con los usuarios que cumplen la condicion de tener las tomas 
#parte2

df1=df.filter(df.IdBeneficiario.isin(lista4)) # elimina las filas de los usuarios que no cumplen en el df de entrada
#lista5=df1.columns #columnas finale 
#df1=df1.withColumn('numNulls1', sum(df1[col].isNull().cast('int') for col in lista5)) # crea una columna contando los nulls por fila 
df1=df1.drop(*lista3)
r=df1.count()
c=df1.columns
notomas=(range(1,len(c),1))
lista6=[]
for c in notomas: # crea una lista con el nombre de las columnas de las tomas que no se consideran 
    name='T'
    if c in lista3:
    #lista6.append(filtrofin(df2, "%s%s"%(name,c))
        lista6.extend(filtrofin(df2, "%s%s"%(name,c) ))

In [0]:
df2=df2.drop(*lista6) # elimina las columnas relacionadas con las tomas que no se tienen en cuenta
display(df2)

In [0]:
# la funcion filtro 4 elimina las columnas asociadas a las tomas no deseadas se debe especificar el # de tomas que se quieren tener en cuenta, 
#luego elimina las columnas que tienen un porcentaje superior de nulls al deseado el cual entra por parametro 
#devuleve un df que segun un porcentaje de nulls predefinido acota la cantidad de columnas 
def filtro4(df,Notomas,porcentaje):
    df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null 
    p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
    p1=p.drop(*filtro2(df, 'IdBeneficiarioT')) #dataframe sin las columnas  IdBeneficiarioT
    listaEPT=p1.columns # lista con las columnas EPT+ Idbeneficiario
    p1=p1.withColumn('numNulls', sum(p1[col].isNull().cast('int') for col in listaEPT)) #crea la columna numNulls que indica la cantidad de nulls en las  columnas EstadoPesoTalla. p1 un data frame con las columnas estado peso el id y la columna numnulls
    f=len(listaEPT) # cantidad de tomas mas el id del ben sin um nulls
    #h=(18-Notomas)
    h=(f-Notomas+2)# esto!
    p1 =p1.filter(p1.numNulls== h) # filtra las columnas segun el no de tomas deseado
    listaEPTf=p1.columns # cantidad de tomas mas el id del ben sin columna num nulls
    
    lista3=listaEPTf[Notomas:]# lista con las columnas no deseadas 
    
    p1=p1.drop(*lista3) # dataframe con las columnas de las EPT deseadas
    colfin=p1.columns
    
    lista4=p1.select('IdBeneficiario').rdd.map(lambda x : x[0]).collect()# hace una lista con los usuarios que cumplen la condicion de tener las tomas 
    #parte2
    
    df1=df.filter(df.IdBeneficiario.isin(lista4)) # elimina las filas de los usuarios que no cumplen en el df de entrada
    #lista5=df1.columns #columnas finale 
    #df1=df1.withColumn('numNulls1', sum(df1[col].isNull().cast('int') for col in lista5)) # crea una columna contando los nulls por fila 
    df1=df1.drop(*lista3)
    r=df1.count()
    c=df1.columns
    Dict_Null = {col:df1.filter(df1[col].isNull()).count() for col in c}# crea un diccionariocon el nombre de la columna y la cantidad de nulls en dicha columna
    val=Dict_Null.values() 
    key=Dict_Null.keys() 
    key1=list(key)#lista con nombre de la columna
    filt=[]
    val1=list(val) # lista con la cantidad de nulls
    for i in range(len(val1)):# calculo del porcentaje de nulls en dicha columna comparado con el numero total de filas 
        if val1[i]/r > porcentaje:# mayor al porcentaje se elimina columna
            filt.append(key1[i])# lista con las columnas que tienen un porcentaje de nulls mayor al entrado por parametro
    #print(filt)
    df2=df1.drop(*filt)# elimina dichas columnas 
    #hasta aca esta bien
    
    notomas=(range(1,Notomas+2,1))
    lista6=[]
    for c in notomas: # crea una lista con el nombre de las columnas de las tomas que no se consideran 
        name='T'
        if c in lista3:
            #lista6.append(filtrofin(df2, "%s%s"%(name,c))
            lista6.extend(filtrofin(df2, "%s%s"%(name,c) ))
    df2=df2.drop(*lista6) # elimina las columnas relacionadas con las tomas que no se tienen en cuenta 
    return df2

In [0]:

def isInt(num):
    try:
        int(num)
        return True
    except ValueError or TypeError:
        return False
    
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError or TypeError:
        return False

def catInt(df): # esta funcion  evalua todas las columnas del df y determina si es int u la guarda en una lsta 
    f=df.first()
    c=df.columns
    inte=[]
    for i in range(0,len(f),1):
        if isinstance(f[i], int)==True and type(f[i])== int:
            inte.append(c[i])
    return(inte)

def catFlo(df): # esta funcion  evalua todas las columnas del df y determina si es float u la guarda en una lsta 
    f=df.first()
    c=df.columns
    #fp=f.topandas()
    flo=[]
    inte=catInt(df)
    for i in range(0,len(f),1):
        if isinstance(f[i], float)==True and type(f[i])== decimal.Decimal :
            flo.append(c[i])
            
    return(flo)
def catstring(df): # esta funcion  evalua todas las columnas del df y determina si es String u la guarda en una lsta 
    f=df.first()
    c=df.columns
    st=[]
    for i in range(0,len(f),1):
        if isinstance(f[i], str)==True and type(f[i])!=int and type(f[i])!=type(Decimal(3.14)) and isInt(f[i])!=True:
            st.append(c[i])
    return(st)

def categorical(df):
    c=df.columns
    catego=[]
    
    for i in range(0,len(c),1):
        if c[i].unique()<50:
              catego.append(c(i))
    return(catego)     

from pyspark.sql.functions import col
# esto funciona bien 

def castype(df): # esta funcion determina la categoria de cada una de las columnas 
    f=df.first() #primera fila del df
    c=df.columns 
    inte=catInt(df)
    flo=catFlo(df)
    st=catstring(df)
    for i in st:
        df.select(col("%s"%(i)).cast('string').alias("%s"%(i)))
    for j in inte:
        df.select(col("%s"%(j)).cast('int').alias("%s"%(j)))
    for k in flo:
        df.select(col("%s"%(k)).cast('float').alias("%s"%(k)))
    return(df,inte,flo,st)



#creacion variable objetivo

## se binariza el historial de los estados peso talla y se elimina la forma categorica de estos 
## Segun el comportamiento de las columnas estado peso talla se crea la columna objetivo que representa el historial de reincidencia

In [0]:
def ept(df):# devuelve el df con las columnas ept mapeados con los codigos
    p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
    eptt=filtro2(df, 'EstadoPesoTalla') # lista de las columnas "estado peso talla" 
    p1=p.drop(*filtro2(p, 'IdBeneficiarioT')) #elimina IdBeneficiarioT
    listaEPT=p1.columns
    soloept=filtro2(p, 'EstadoPesoTalla')
    listaEPT=len(soloept)
    
    def somefunc(value):
        if value== 'Peso adecuado para la talla': 
                  return int(0)
        elif value=='Riesgo de desnutrición aguda':
            return int(0)
        elif value=='Desnutricion Aguda Moderada':
            return int(1)
        elif value=='Riesgo de Sobrepeso':
            return int(0)
        elif value=='Desnutricion aguda severa':
            return int(1)
        elif value=='Obesidad':
            return int(0)
        elif value=='Sobrepeso':
            return int(0)
        else:
            return int(0)
    udfsomefunc = F.udf(somefunc, IntegerType())
    EPTs=[]
    for c in (range(1,listaEPT+1)):
         EPTs.append('EPT%s'%(c))
    for i in range(listaEPT):
        df= df.withColumn(EPTs[i], udfsomefunc(df[soloept[i]]))
        #df= df.drop(*eptt)
    return   df

In [0]:
def reincidenciaff2(df):#
    soloept=filtro2(df,'EPT')# se crea una lista con las columnas estado peso talla binario
    c=df.select(soloept) #se crea el data frame con dichas columnas 
    listu = c.collect() #se obtienen todas las filas de dicho df 
    s=len(listu[1])# la cantidad de columnas 
    x=range(s,0,-1)
    l=len(listu) #la dimension de la matriz 
    t=range(0,l,1)
    ant=0
    rein=[]
    for j in t :
        r1 = False # formatea el estado de la bandera para cada fila
        for i in x: # se recorre las filas 

            if i==s:# primera iteracion de la ultima a la primera

                ant=listu[j][i-1] # primera iteracion ant = al valor ultima toma

            else:
                #print(0)
                if ant!=listu[j][i-1]:# si el anterior es diferente al actual y si r1 = true : r2=true y hay incidencia 
                    #print(ant)

                    if r1== True :
                        rein.append(1)
                        break  
                    ant=listu[j][i-1] # se actualiza el anterior
                    r1=True
            if s-i==s-1:# si el ultimo elemento es igual al penultimo elemento
                rein.append(0)
                break
            
    
    
    Ids=df.select('IdBeneficiario').rdd.flatMap(lambda x: x).collect() # se obtiene el Id de los usuarios como lista
    

    daa={'IdBeneficiarion': Ids, 'Reincidencia': rein } #se unen ambas listas el id y el estado de reincidencia
    dfp=pd.DataFrame(daa)# se crea el dataframe en pandas
    mySchema = StructType([StructField("IdBeneficiarion", StringType(), True)\
                           , StructField("Reincidencia",IntegerType(),True)])# se establece el schema

    dff = spark.createDataFrame(dfp, schema=mySchema)# se crea el dataframe en spark 


    merged_df=df.join(dff,df.IdBeneficiario ==  dff.IdBeneficiarion,"inner") # se hace el join entre el df que entra y el df que contiene el estado de reincidencia bajo la llave del ID 
    merged_df=merged_df.drop(merged_df.IdBeneficiarion) # se elimina la columna duplicada del ID 
    return(merged_df)



In [0]:
#esta funcion :
#llena los campos vacios con nulls
#crea la columna reincidencia
#crea la columna desnutricion
# crea la columna Tomas Originales**
#separa el dataframe entre reincidentes y no reincidentes 
#llena los nulls de reincidentes con los valores mas frecuentes 
#llena los nulls de Noreincidentes con los valores mas frecuentes 
#une los dosdataframes y entrega la union
#fal
def schemaandnulls(df):# esta funcion arregla el dataframe llenando valores nulls
    df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null
    df1=ept(df)
    df2=reincidenciaff2(df1)
    df3=df2.filter(df2.Reincidencia==1)
    
    dic=nulls(df3)
    for k, v  in dic.items():
    
        if v!=0.0 | v!=0:
            
            mode=df3.groupBy(k).count().sort(col("count").desc()).collect()[0][0]
            df3 = df3.na.fill({k:mode})
        
    
    df4=df2.filter(df2.Reincidencia==0)
    
    dic2=nulls(df4)
    for k2, v2  in dic2.items():
    
        if v2!=0.0 | v2!=0:
            
            mode=df4.groupBy(k2).count().sort(col("count").desc()).collect()[0][0]
            df4 = df4.na.fill({k2:mode})
    
    rec=df3.unionByName(df4, allowMissingColumns=True)
        
    return(rec)

#actualizacion sabana descriptiva sin nulls

In [0]:
raw_data1=schemaandnulls(raw_datax)
    #---------------------------- Borrar Histórico ----------------------------------------#
    #dbfsWriteDelta2 = "dbfs:/dbfs/gold"
    dbfsWriteDelta2="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabanafinal6/"
    try:
        dbutils.fs.ls(dbfsWriteDelta2)
        print(" *** Borrando Directorio *** ")
        files = dbutils.fs.ls(dbfsWriteDelta2)
        for f in files:
            f.path
            dbutils.fs.rm(f.path, recurse=True)
    except:
        print(" *** No Existe Directorio *** ")
        pass
    #---------------------------- Directorio de Salida ------------------------------------#
    print(" *** Guardando en el DataLake *** ")
    pathSave = dbfsWriteDelta2
    nombreTabla = "dbo.raw_data1"

    baseprueb.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{pathSave}") 
    print(" *** Guardado en el DataLake *** ")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathSave}/'")

In [0]:
def balancing(df):# funcion que balancea 
    balancingRatio = df.filter(col("Reincidencia") == 1).count() / df.count()
    calculateWeights = udf(lambda x: 1 * balancingRatio if x == 0 else (1 * (1.0 - balancingRatio)), DoubleType())
    weightedDataset = df.withColumn("Reincidenciaw", calculateWeights("Reincidencia"))
    return(weightedDataset)

In [0]:
poc=filtro4(raw_datax,17,0.1)
pathpoc="dbfs:/dbfs/mnt/poc17.parquet"
poc.write.mode("overwrite").parquet(pathpoc)

Balanceo del dataframe

In [0]:
path="dbfs:/dbfs/mnt/p17rfinal.parquet"
prueba = spark.read.options(inferSchema='True',header='True').format("parquet").load(path)

In [0]:

def reincidentes(df):# esta funcion devuelve el dataframe de solo los sujetos reincidentes 
    #df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null
    df1=ept(df)
    df2=reincidenciaff2(df1)
    df3=df2.filter(df2.Reincidencia==1)
    
    df4=df2.filter(df2.Reincidencia==0)
    return(df3)


#### Etapa de String indexer OneHot encoding y vector assembler

Objetivo: volver las variables tipo string a una forma numérica y todas las categoricas hacerles OHE para crear el vector features

Definir la pipeline de ML
 como con la mayoría de las aplicaciones de ML, debe realizar un preprocesamiento de los datos antes de poder usarlos para entrenar un modelo. MLlib proporciona canalizaciones que le permiten combinar varios pasos en un solo flujo de trabajo. 
 
 Pipeline se usa para combinar dos pasos:

StringIndexer convierte las etiquetas de valores numéricos a valores categóricos.

In [0]:
def features(df): # esta funcion le entra un df y crea un nuevo dataframe con el OHE de las columnas categoricas y devuelve un df con el el vector features que entra a los modelos de ML 
    
    x=catstring(df)
    lista=[]
    listinx=[]
    for i in x :
        lista.append("num_"+str(i))# se crea la lista con el nombre de las columnas que se vuelven de string a num
    y=lista
    stringindexer_stages =[StringIndexer(inputCols= x, outputCols= y).setHandleInvalid("skip")] # toma todas las columnas que son string 
    listunt=catInt(df) # lista con las columnas que son int 
    #listunt.remove("Reincidencia")
    listaohc=[]
    for i in y :# creacion de los nombrede las columnas a las que se les hace OHE
            listaohc.append("vec"+str(i))
    
    onehotencoder_stages = [OneHotEncoder( inputCols=y, outputCols=listaohc, handleInvalid='error')]#, dropLast=True)
    vectorassembler_stage = VectorAssembler(inputCols=listaohc, outputCol='features')
    all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
    pipeline = Pipeline(stages=all_stages).fit(df).transform(df)
    final_columns = ["Idbeneficiario"]+['features'] + ['Reincidenciaw']+['Reincidencia'] #listunt +
    df_features = pipeline.select(final_columns)
    return (df_features)

# construccion de las 3 sabanas de datos:
- La primera con todas las variables 
- La segunda con el 75% de las variables 
- La tercera con el 50% de las variables

In [0]:
p1705=filtro4(raw_datax,17,0)
p1005=filtro4(raw_datax,10,0.2)
p605=filtro4(raw_datax,6,0.3)

#creacion de las columnas ept 
p17ept=ept(p1705)
p1005ept=ept(p1005)
p605ept=ept(p605)

# creacion de la columna reincidencia
p1705r=reincidenciaff2(p17ept)
p1005eptr=reincidenciaff2(p1005ept)
p605eptr=reincidenciaff2(p605ept)

#arreglar schema categorizando cada una
p1705rcast=castype(p1705r)
p1005rcast=castype(p1005eptr)
p605cast=castype(p605eptr)

p1705rcast=balancing(p1705rcast)
p1005rcast=balancing(p1005rcast)
p605cast=balancing(p605cast)
#Creacion del dataframe final con la columna features que contiene la informacion de todas las columnas 


path_to_datap1705r="dbfs:/dbfs/mnt/p1705r.parquet"
path_to_datap605eptr="dbfs:/dbfs/mnt/p605eptr.parquet"
path_to_datap1005eptr="dbfs:/dbfs/mnt/p1005eptr.parquet"

p1705rcast.write.mode("overwrite").parquet(path_to_datap1705r)
p1005rcast.write.mode("overwrite").parquet(path_to_datap605eptr)
p605cast.write.mode("overwrite").parquet(path_to_datap1005eptr)

In [0]:
path_to_datap1705r="dbfs:/dbfs/mnt/p1705r.parquet"
path_to_datap605eptr="dbfs:/dbfs/mnt/p605eptr.parquet"
path_to_datap1005eptr="dbfs:/dbfs/mnt/p1005eptr.parquet"

p1705rcast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap1705r)
p605cast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap605eptr)
p1005rcast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap1005eptr)

In [0]:
p17feat=features(p1705rcast)
p6feat=features(p605cast)
p10feat=features(p1005rcast)

path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"

p17feat.write.mode("overwrite").parquet(path_17feat)
p10feat.write.mode("overwrite").parquet(path_10feat)
p6feat.write.mode("overwrite").parquet(path_6feat)

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"

p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
path_6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)

In [0]:
print(p1705rcast.count())
print(p605cast.count())
print(p1005rcast.count())

In [0]:
nulls(p1705rcast)

In [0]:
nulls(p605cast)

In [0]:
nulls(p1005rcast)

%md tres modelos de machine learning 

- regresion Lineal
- Regresion Logistica
- random Forest

In [0]:
%sh

pip install mlflow

/databricks/python_shell/scripts/PythonShell.py configure

In [0]:
import mlflow
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

metricName: pyspark.ml.param.Param[MulticlassClassificationEvaluatorMetricType] = Param(parent='undefined', name='metricName', doc='metric name in evaluation (f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss)')

# Revisar los resultados registrados

Para ver el experimento de MLflow asociado con el cuaderno, haga clic en el ícono Experimento en la barra de contexto del cuaderno en la parte superior derecha. Todas las ejecuciones del cuaderno aparecen en la barra lateral. Para comparar más fácilmente sus resultados, haga clic en el ícono en el extremo derecho de Ejecuciones de experimentos (muestra "Ver IU de experimento" cuando pasa el mouse sobre él). Aparece la página Experimento.

mas informacion en https://docs.microsoft.com/es-mx/azure/databricks/_static/notebooks/mllib-mlflow-integration.html

#### Regresión lineal

In [0]:
def reglin(feat):    
    model_dff1=feat
    train_df,test_df=model_dff1.randomSplit([0.8,0.2])
    lin_Reg=LinearRegression(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    lrparamGrid = (ParamGridBuilder()
             .addGrid(lin_Reg.regParam, [ 0.1, 0.5, 1.0, 2.0])

             .addGrid(lin_Reg.elasticNetParam, [0.0, 0.5, 0.75, 1.0])

             .addGrid(lin_Reg.maxIter, [1, 5, 10, 20, 30])

             .build())
    evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Reincidencia", weightCol="Reincidenciaw")
    #evaluator = BinaryClassificationEvaluator( labelCol='Reincidencia',weightCol="Reincidenciaw")
    cv_LiR = CrossValidator(estimator = lin_Reg, 
                      estimatorParamMaps = lrparamGrid,
                      evaluator = evaluator,
                      numFolds = 5)
    
    with mlflow.start_run():
  # Run the cross validation on the training dataset. The cv.fit() call returns the best model it found.
  # If you omit mlflow.start_run(), then this cell could run once, but a second run would hit conflicts when attempting to overwrite the first run.

        crossvalidation_mod = cv_LiR.fit(train_df)# fiting cross validation 
    #lrcvSummary = crossvalidation_mod.bestModel.summary # resumen de los parametros de mejor modelo
    #prediccion en el conjunto de entrenamiento
    
    # Log the best model.
    mlflow.spark.log_model(spark_model=crossvalidation_mod.bestModel, artifact_path='best-model') 
    
    pred_train = crossvalidation_mod.transform(train_df)

    #data frame con la prediccion en el conjunto de test
    lrpredictions =crossvalidation_mod.transform(test_df) # evaluacion del modelo 

    
    #label_pred_train = pred_train.select('Idbeneficiario','Reincidencia', 'prediction')
    
    label_pred_test = lrpredictions.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')


    
    
    #Confusion matrix from test data
    tp = label_pred_test[(lrpredictions.Reincidencia == 1) & (lrpredictions.prediction > 0.5)].count()
    tn = label_pred_test[(lrpredictions.Reincidencia == 0) & (lrpredictions.prediction< 0.5)].count()
    fp = label_pred_test[(lrpredictions.Reincidencia == 0) & (lrpredictions.prediction > 0.5)].count()
    fn = label_pred_test[(lrpredictions.Reincidencia == 1) & (lrpredictions.prediction < 0.5)].count()
    
    #indicadores
    coef=evaluator.evaluate(lrpredictions)
    r2=evaluator.setMetricName('r2').evaluate(lrpredictions)# R^2
    coef=print('Intercept: ', crossvalidation_mod.bestModel.intercept)
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    
   
    
    
     #evaluacion rmse
    test_rmse = evaluator.setMetricName('rmse').evaluate((lrpredictions))
    mlflow.log_metric('test_rmse', test_rmse)
    
     #evaluacion mae
    test_mae = evaluator.setMetricName('mae').evaluate((lrpredictions))
    mlflow.log_metric('test_mae' , test_mae)
    
    #evaluacion r2
    test_r2 = evaluator.setMetricName('r2').evaluate((lrpredictions))
    mlflow.log_metric('test_r2' , test_r2)


    print("MAE: ", test_mae)
    print("R-squared: ", test_r2)
    print(score)
    print('RMSE:',test_rmse)
    mlflow.end_run()
    return(label_pred_test,lin_Reg)



In [0]:
mlflow.end_run()

In [0]:
pip install databricks-sql-connector

Python interpreter will be restarted.
Requirement already satisfied: databricks-sql-connector in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (2.0.2)
Requirement already satisfied: pyarrow>=5.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from databricks-sql-connector) (8.0.0)
Requirement already satisfied: thrift>=0.13.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from databricks-sql-connector) (0.16.0)
Requirement already satisfied: pandas>=1.3.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/lib/python3.8/site-packages (from databricks-sql-connector) (1.4.3)
Requirement already satisfied: pytz>=2020.1 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.3.0->databricks-sql-connector) (2020.5)
Requirement already satisfied: numpy>=1.18.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.3.0->databricks-sql-connector) (1.19.2)
Requirement already satisfied: python-dateutil>=2.8.1 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.3.0->databricks-sql-connector) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.8.1->pandas>=1.3.0->databricks-sql-connector) (1.15.0)
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0b69db2b-837a-4b7b-adf4-8435a415247f/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
mlflow.spark.log_model(model, "LinReg")

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17reglin=reglin(p17feat)
path_17reglin="dbfs:/dbfs/mnt/p17reglin.parquet"
p17reglin[0].write.mode("overwrite").parquet(path_17reglin)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Intercept: 0.4309582154772112
MAE: 0.36601645152936585
R-squared: -1.0977664559005516
{'tp': 402, 'tn': 6855, 'fp': 406, 'fn': 163, 'accuracy': 0.9272936365959622, 'recall': 0.7115044247787611, 'precision': 0.4975247524752475, 'f1': 0.5855790240349601}
RMSE: 0.3033816028987365

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-475208278801497> in <module> 
 3 p17reglin = reglin ( p17feat ) 
 4 path_17reglin = "dbfs:/dbfs/mnt/p17reglin.parquet" 
 ----> 5 p17reglin . write . mode ( "overwrite" ) . parquet ( path_17reglin ) 

 AttributeError : 'tuple' object has no attribute 'write'

In [0]:
p17reglin[0].write.mode("overwrite").parquet(path_17reglin)
p17reglin[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Lineal_Reg/p17reglin/')

In [0]:
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p6reglin=reglin(p6feat)
path_6reglin="dbfs:/dbfs/mnt/p6reglin.parquet"
p6reglin[0].write.mode("overwrite").parquet(path_6reglin)
p6reglin[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Lineal_Reg/p6reglin/') # asi se guarda el modelo final

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Intercept: 0.3784177962770636
MAE: 0.3187360048958478
R-squared: 0.54504794473559
{'tp': 1335, 'tn': 50815, 'fp': 320, 'fn': 133, 'accuracy': 0.9913883238598559, 'recall': 0.909400544959128, 'precision': 0.8066465256797583, 'f1': 0.8549471661863591}
RMSE: 0.33721431602201263

In [0]:
mlflow.end_run()
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p10reglin=reglin(p10feat)
path_10reglin="dbfs:/dbfs/mnt/p10reglin.parquet"
p10reglin[0].write.mode("overwrite").parquet(path_10reglin)
p10reglin[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Lineal_Reg/p10reglin/') # asi se guarda el modelo final

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Intercept: 0.2053093725993094
MAE: 0.1274896476892479
R-squared: 0.8960572516809647
{'tp': 230, 'tn': 67569, 'fp': 402, 'fn': 2, 'accuracy': 0.9940765068985236, 'recall': 0.9913793103448276, 'precision': 0.3639240506329114, 'f1': 0.5324074074074074}
RMSE: 0.1600587509235865

#### Regresion Logística

In [0]:
from pyspark.ml.classification import LogisticRegression
def reglogist(feat):
    #se toman solo las columnas features y reincidencia balanceado
    model_df=feat
    
    #divison aleatoria
    training_df,test_df=model_df.randomSplit([0.8,0.2]) 
    
    #aplicacion de la regresion logistica 
    log_reg = LogisticRegression(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    
   
    #evaluador regresion 
    evaluator_LR = RegressionEvaluator(predictionCol="prediction", labelCol="Reincidencia", weightCol="Reincidenciaw")
    # Creacion ParamGrid para Cross Validation
    paramGrid_LR = (ParamGridBuilder()
                 .addGrid(log_reg.regParam, [ 0.1, 0.5, 1.0, 2.0])
                 .addGrid(log_reg.elasticNetParam, [0.0, 0.5, 0.75, 1.0])
                 .addGrid(log_reg.maxIter, [1, 5, 10, 20, 40])
                 .build())


    pipelinelr = Pipeline(stages=[log_reg])

    # Crea 5-fold CrossValidator
    cv_LR = CrossValidator(estimator = pipelinelr,
                          estimatorParamMaps = paramGrid_LR,
                          evaluator = evaluator_LR,
                          numFolds = 5)
    
    
    with mlflow.start_run(run_name='Reg_Log'):
  # Run the cross validation on the training dataset. The cv.fit() call returns the best model it found.
  # If you omit mlflow.start_run(), then this cell could run once, but a second run would hit conflicts when attempting to overwrite the first run.

    
    # Fit cross-validation model Run cross validations.
        cvModel_lr = cv_LR.fit(training_df)
    
     #. monitoreao del progreso de los parametros.
    mlflow.spark.log_model(spark_model=cvModel_lr.bestModel, artifact_path='best-model')
    
    #Prediccion
    
    #Prediccion en data de entreno 
    pred_train = cvModel_lr.transform(training_df)
    
    #Prediccion en data de test 
    predictions_lr = cvModel_lr.transform(test_df)
    
    
    #Confusion matrix from test data

    
    label_pred_test = predictions_lr.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')
    
    
    # Evaluacion del  modelo indicadores
    
    tp = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction > 0.5)].count()
    tn = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction< 0.5)].count()
    fp = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction > 0.5)].count()
    fn = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction < 0.5)].count()
    
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    
    
    
    
    
    
    #evaluacion rmse
    test_rmse = evaluator_LR.setMetricName('rmse').evaluate(predictions_lr)
    mlflow.log_metric('test_rmse', test_rmse)
    
     #evaluacion mae
    test_mae = evaluator_LR.setMetricName('mae').evaluate(predictions_lr)
    mlflow.log_metric('test_mae' , test_mae)
    
    #evaluacion r2
    test_r2 = evaluator_LR.setMetricName('r2').evaluate(predictions_lr)
    mlflow.log_metric('test_r2' , test_r2)
    
    
    
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    print(score)
    print("MAE: ", test_mae)
    print("R-squared: ", test_r2)
    print('RMSE:',test_rmse)
    
    
    

    
    return(label_pred_test,log_reg)





In [0]:
mlflow.end_run()
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17logreg=reglogist(p17feat)
path_17logreg="dbfs:/dbfs/mnt/p17logreg.parquet"
p17logreg[0].write.mode("overwrite").parquet(path_17logreg)
p17logreg[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Logisitc_Reg/p17logreg/')



In [0]:
mlflow.end_run()
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p6logreg=reglogist(p6feat)
path_6logreg="dbfs:/dbfs/mnt/p6logreg.parquet"
p6logreg[0].write.mode("overwrite").parquet(path_6logreg)
p6logreg[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Logisitc_Reg/p6logreg/')


MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
{'tp': 1330, 'tn': 50834, 'fp': 548, 'fn': 107, 'accuracy': 0.9875991593934002, 'recall': 0.9255393180236604, 'precision': 0.7082002129925452, 'f1': 0.8024132730015083}
MAE: 0.04194310212107543
R-squared: 0.8321341996735919
RMSE: 0.20480015166272564

In [0]:
mlflow.end_run()

path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p10logreg=reglogist(p10feat)
path_10logreg="dbfs:/dbfs/mnt/p10logreg.parquet"
p10logreg[0].write.mode("overwrite").parquet(path_10logreg)
p10logreg[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Logisitc_Reg/p10logreg/')



MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
{'tp': 232, 'tn': 67726, 'fp': 409, 'fn': 5, 'accuracy': 0.9939448897209384, 'recall': 0.9789029535864979, 'precision': 0.3619344773790952, 'f1': 0.5284738041002278}
MAE: 0.012757788456952735
R-squared: 0.9483462992183667
RMSE: 0.11295038050822466

#### Random Forest

In [0]:
def randomforest(feat):
    model_RF=feat
    
    #division aleatoria 
    train_df,tes_df=model_RF.randomSplit([0.8,0.2])
    
    #estimador
    random_forest = RandomForestClassifier(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    
    
    # Evaluate model
    
    evaluator = MulticlassClassificationEvaluator( labelCol='Reincidencia',weightCol="Reincidenciaw")
   
    # Creacion grid de parametrospara Cross Validation

    pipeline = Pipeline(stages=[random_forest])

    rfparamGrid = (ParamGridBuilder()
               .addGrid(random_forest.maxDepth, [ 5, 10, 20])
               .addGrid(random_forest.maxBins, [ 50])
               .addGrid(random_forest.numTrees, [ 50])
               .addGrid(random_forest.minInfoGain, [0.0, 0.1, 0.2, 0.3])
             .build())



    #  5-fold CrossValidator
    rfcv = CrossValidator(estimator = pipeline,
                          estimatorParamMaps = rfparamGrid,
                          evaluator = evaluator,
                          numFolds = 5)
    
    
     
    with mlflow.start_run(run_name='Random_Forest'):
  # Run the cross validation on the training dataset. The cv.fit() call returns the best model it found.
  # If you omit mlflow.start_run(), then this cell could run once, but a second run would hit conflicts when attempting to overwrite the first run.

    
    #Fit cross-validation model
        crossvalidation_mod = rfcv.fit(train_df)
    
    
    # Log the best model. monitoreao del progreso de los parametros.
    mlflow.spark.log_model(spark_model=crossvalidation_mod.bestModel, artifact_path='best-model')
    
    
    
    #prediction
    
    #en training
    pred_train = crossvalidation_mod.transform(train_df)
    
    #en test
    pred_test = crossvalidation_mod.transform(tes_df)
    
    
    label_pred_test = pred_test.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')
    
    #Prediction performance
    
    # Evaluacion del  modelo indicadores
    
    tp = label_pred_test[(pred_test.Reincidencia == 1) & (pred_test.prediction > 0.5)].count()
    tn = label_pred_test[(pred_test.Reincidencia == 0) & (pred_test.prediction< 0.5)].count()
    fp = label_pred_test[(pred_test.Reincidencia == 0) & (pred_test.prediction > 0.5)].count()
    fn = label_pred_test[(pred_test.Reincidencia == 1) & (pred_test.prediction < 0.5)].count()
    
    
    
    #evaluacion de la precision ponderada
    
    
    
 
     #evaluacion hammingLoss
    test_hammingLoss = evaluator.setMetricName('hammingLoss').evaluate(pred_test)
    mlflow.log_metric('test_hammingLoss' , test_hammingLoss)
    
    #evaluacion hammingLoss
    test_precision = evaluator.setMetricName('weightedPrecision').evaluate(pred_test)
    mlflow.log_metric('test_precision' , test_precision)
    
    
    print("hammingLoss: ", test_hammingLoss)
    print("rf_precision: ", test_precision)
    
    
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    print("Indicadores: ",score)
    
    
    mlflow.end_run()
    
    return(label_pred_test,random_forest)
    
    
    
    
   
    

In [0]:
mlflow.end_run()
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17rf=randomforest(p17feat)
path_17rf="dbfs:/dbfs/mnt/p17rf.parquet"
p17rf[0].write.mode("overwrite").parquet(path_17rf)
p17rf[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p17rf/')

In [0]:

mlflow.end_run()
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load( path_6feat )
p6rf=randomforest(p6feat)
path_6rf="dbfs:/dbfs/mnt/p6rf.parquet"

p6rf[0].write.mode("overwrite").parquet(path_6rf)
p6rf[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p6rf/')



MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
2022/07/19 23:47:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzpcn8avl/model, flavor: spark), fall back to return ['pyspark==3.1.2']. Set logging level to DEBUG to see the full traceback.
hammingLoss: 0.16055854502057276
rf_precision: 0.8462151731847736
Indicadores: {'tp': 1114, 'tn': 46286, 'fp': 4522, 'fn': 339, 'accuracy': 0.9069860890530224, 'recall': 0.76668960770819, 'precision': 0.19765791341376862, 'f1': 0.3142897446748483}

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-475208278801508> in <module> 
 4 p6rf = randomforest ( p6feat ) 
 5 path_6rf = "dbfs:/dbfs/mnt/p6rf.parquet" 
 ----> 6 p6rf . write . mode ( "overwrite" ) . parquet ( path_6rf ) 
 7 p6rf [ 0 ] . write . mode ( "overwrite" ) . parquet ( path_6rf ) 
 8 p6rf [ 1 ] . write ( ) . overwrite ( ) . save ( 'dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p6rf/' ) 

 AttributeError : 'tuple' object has no attribute 'write'

In [0]:
path_6rf="dbfs:/dbfs/mnt/p6rf.parquet"

p6rf[0].write.mode("overwrite").parquet(path_6rf)
p6rf[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p6rf/')

In [0]:

mlflow.end_run()
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p10rf=randomforest(p10feat)
path_10rf="dbfs:/dbfs/mnt/p10rf.parquet"
p10rf[0].write.mode("overwrite").parquet(path_10rf)
p10rf[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p10rf/')



MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
2022/07/20 01:04:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpiyr570ky/model, flavor: spark), fall back to return ['pyspark==3.1.2']. Set logging level to DEBUG to see the full traceback.
hammingLoss: 0.04198839858810897
rf_precision: 0.9593147605303869
Indicadores: {'tp': 228, 'tn': 67319, 'fp': 853, 'fn': 19, 'accuracy': 0.9872550022654526, 'recall': 0.9230769230769231, 'precision': 0.21091581868640147, 'f1': 0.3433734939759036}

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3428115858088594> in <module> 
 5 path_10rf = "dbfs:/dbfs/mnt/p10rf.parquet" 
 6 p10rf [ 0 ] . write . mode ( "overwrite" ) . parquet ( path_10rf ) 
 ----> 7 p10rf [ 1 ] . write ( ) . overwrite ( ) . save ( 'dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p10rf/' ) 
 8 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 306 if not isinstance ( path , str ) : 
 307 raise TypeError ( "path should be a string, got type %s" % type ( path ) ) 
 --> 308 self . _jwrite . save ( path ) 
 309 
 310 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o97294.save.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: hadoop_azure_shaded.com.microsoft.azure.storage.StorageException: This operation is not permitted on a non-empty directory.
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.delete(AzureNativeFileSystemStore.java:2828)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.delete(AzureNativeFileSystemStore.java:2838)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.deleteFile(NativeAzureFileSystem.java:2271)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem$1.execute(NativeAzureFileSystem.java:2219)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureFileSystemThreadPoolExecutor.executeParallel(AzureFileSystemThreadPoolExecutor.java:223)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.deleteWithInstrumentation(NativeAzureFileSystem.java:2229)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.delete(NativeAzureFileSystem.java:1960)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.delete(NativeAzureFileSystem.java:1933)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$delete$3(DatabricksFileSystemV2.scala:766)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$delete$2(DatabricksFileSystemV2.scala:764)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$withUserContextRecorded$2(DatabricksFileSystemV2.scala:1013)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:261)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:258)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:305)

AUC ranges in value from 0 to 1. A model whose predictions are 100% wrong has an AUC of 0.0; one whose predictions are 100% correct has an AUC of 1.0.

AUC is desirable for the following two reasons:

AUC is scale-invariant. It measures how well predictions are ranked, rather than their absolute values.
AUC is classification-threshold-invariant. It measures the quality of the model's predictions irrespective of what classification threshold is chosen.

In [0]:
path_10rf="dbfs:/dbfs/mnt/p10rf.parquet"
p10rf[0].write.mode("overwrite").parquet(path_10rf)
p10rf[1].write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p10rf/')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1313497070338360> in <module> 
 1 path_10rf = "dbfs:/dbfs/mnt/p10rf.parquet" 
 2 p10rf [ 0 ] . write . mode ( "overwrite" ) . parquet ( path_10rf ) 
 ----> 3 p10rf [ 1 ] . write ( ) . overwrite ( ) . save ( 'dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/p10rf/' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 306 if not isinstance ( path , str ) : 
 307 raise TypeError ( "path should be a string, got type %s" % type ( path ) ) 
 --> 308 self . _jwrite . save ( path ) 
 309 
 310 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o97381.save.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: hadoop_azure_shaded.com.microsoft.azure.storage.StorageException: This operation is not permitted on a non-empty directory.
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.delete(AzureNativeFileSystemStore.java:2828)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.delete(AzureNativeFileSystemStore.java:2838)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.deleteFile(NativeAzureFileSystem.java:2271)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem$1.execute(NativeAzureFileSystem.java:2219)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureFileSystemThreadPoolExecutor.executeParallel(AzureFileSystemThreadPoolExecutor.java:223)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.deleteWithInstrumentation(NativeAzureFileSystem.java:2229)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.delete(NativeAzureFileSystem.java:1960)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.delete(NativeAzureFileSystem.java:1933)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$delete$3(DatabricksFileSystemV2.scala:766)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$delete$2(DatabricksFileSystemV2.scala:764)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$withUserContextRecorded$2(DatabricksFileSystemV2.scala:1013)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:261)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:258)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:305)
	at

# Se actualizacion y carga en synapse 

La siguiente funcion tiene el objetivo de cargar las sabanas de datos que contienen las predicciones por sabana y por modelo 
en esta funcion se puede escoger la sabana y el modelo que se desee visualizar y estabelcer el enlace donde se debe guardar para poder leerlo con el power BI 
- Actualmente los datos visualizados vienen de p10logreg que es el resultado de pasar la sabana con 10 tomas por el modelo de regresion logistica

In [0]:
#path_grafica="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoGrafica/" # el path que lee el PWBI
#pathpredic= "dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoPredicciones/" #path de dataframe prediccion
#pathsabana="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabana6/" #path de dataframe con toda la informacion 
#pathsabana="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabanafinal/"#prueba
#dfmodelo = p10logreg.select('IdBeneficiario','prediction')#6feat


def final(dfsabana,dfmodelo,pathsabana, pathpredic,path_grafica):
    dfPredicciones = dfmodelo.select('IdBeneficiario','prediction')
    
    #descriptiva
    #---------------------------- Borrar Histórico ----------------------------------------#
    
    
    try:
        dbutils.fs.ls(pathsabana)
        print(" *** Borrando Directorio *** ")
        files = dbutils.fs.ls(pathsabana)
        for f in files:
            f.path
            dbutils.fs.rm(f.path, recurse=True)
    except:
        print(" *** No Existe Directorio *** ")
        pass
    #---------------------------- Directorio de Salida ------------------------------------#
    pathSave = pathsabana
    nombreTabla = "dbo.dfsabana"
    dfsabana.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{pathSave}") 
    print(" *** Guardado en el DataLake *** ")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathsabana}/'")
    
    # predicciones 

        #---------------------------- Borrar Histórico ----------------------------------------#
    
    try:
        dbutils.fs.ls(pathpredic)
        print(" *** Borrando Directorio *** ")
        files = dbutils.fs.ls(pathpredic)
        for f in files:
            f.path
            dbutils.fs.rm(f.path, recurse=True)
    except:
        print(" *** No Existe Directorio *** ")
        pass
    #---------------------------- Directorio de Salida ------------------------------------#
    
    nombreTabla = "dbo.dfPredicciones"
    dfPredicciones.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{pathpredic}") 
    print(" *** Guardado en el DataLake *** ")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathpredic}/'")
    #fin
    
    #reconstruccion sabana descriptiva y predictiva
    BaseDescriptivaPwBi = spark.read.format("delta").load(pathsabana)
    BasePrediccionesPwBi = spark.read.format("delta").load(pathpredic)
    BaseDescriptivaPwBi.createOrReplaceTempView("BaseDescriptivaPwBi")
    BasePrediccionesPwBi.createOrReplaceTempView("BasePrediccionesPwBi")
    
    # cruce entre bases descriptiva y predicciones
    BaseGrafica = spark.sql("""
    SELECT
        BP.Idbeneficiario as Idben1,
        BP.prediction as prediccion,
        DB.*,LEFT(CAST((NOW() - INTERVAL 5 HOUR) AS STRING),16) AS Fecha_Consulta

    FROM
        BasePrediccionesPwBi BP
        INNER JOIN BaseDescriptivaPwBi DB 
        ON BP.IdBeneficiario = DB.IdBeneficiario
    """)
    BaseGrafica.createOrReplaceTempView("BaseGrafica")
    
    #escritura de base grafica 
    
    #---------------------------- Borrar Histórico ----------------------------------------#
    
    try:
        dbutils.fs.ls(path_grafica)
        print(" *** Borrando Directorio *** ")
        files = dbutils.fs.ls(path_grafica)
        for f in files:
            f.path
            dbutils.fs.rm(f.path, recurse=True)
    except:
        print(" *** No Existe Directorio *** ")
        pass
    #---------------------------- Directorio de Salida ------------------------------------#
    
    nombreTabla = "dbo.BaseGrafica"
    BaseGrafica.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{path_grafica}") 
    print(" *** Guardado en el DataLake *** ")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{path_grafica}/'")
    return(BaseGrafica)

In [0]:
#path_10logreg="dbfs:/dbfs/mnt/p10logreg.parquet"
path_6logreg="dbfs:/dbfs/mnt/p6logreg.parquet"
pathpredic="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoPredicciones6/"
dfmodelo=spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6logreg)
pathsabana="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticosabana6/" #path de dataframe con toda la informacion
path_to_data="dbfs:/dbfs/gold"
dfsabana=spark.read.options(inferSchema='True',header='True').format("delta").load(path_to_data)
path_grafica="dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoGrafica6/"

display(final(dfsabana,dfmodelo,pathsabana, pathpredic,path_grafica))

*** Borrando Directorio *** 
 *** Guardado en el DataLake *** 
 *** Borrando Directorio *** 
 *** Guardado en el DataLake *** 
 *** Borrando Directorio *** 
 *** Guardado en el DataLake ***

Idben1,prediccion,IdBeneficiario,CodDocumento,NumeroDocumento,PrimerNombre,SegundoNombre,PrimerApellido,SegundoApellido,FechaNacimientoBeneficiario,sexoBeneficiario,NombreSexo,RegimenSeguridadSocial,CodPaisNacimientoBeneficiario,NombPaisNacimientoBeneficiario,CodPaisRasidenciaBen,NombPaisRasidenciaBen,CodDepartamentoResidenciaBen,NomDepartamentoResidenciaBen,CodMunicipResidenciaBen,NomMunicipResidenciaBen,CodComunaResidenciaBen,NomComunaResidenciaBen,CodBarrioResidenciaBen,NomBarrioResidenciaBen,CodVeredaResidenciaBen,NomVeredaResidenciaBen,CodRancheriaResidenciaBen,NomRancheriaResidenciaBen,PresentaDiscapacidad,ZonaUbicacionBeneficiario,GrupoEtnico,MesesLactanciaMaternaExclusiva,MesesLactanciaMaternaTotal,PesoBeneficiarioAlNacer,TallaBeneficiarioAlNacer,AntecendentePremadurez,EdadGestacionarAlNacer,TomaT1,NoTomaT1,IdDireccionesICBFT1,DireccionICBFT1,NombreServicioT1,EdadBeneficiarioDiasT1,FechaRegistroSaludNutricionT1,FechaMedicionPerimetroBraquialT1,MedicionPerimetroBraquialT1,pesoT1,TallaT1,ZScoreTallaEdadT1,ZScorePesoEdadT1,ZScorePesoTallaT1,ZScoreIMCT1,EstadoTallaEdadT1,EstadoPesoEdadT1,EstadoPesoTallaT1,EstadoIMCT1,FlagT1,PresentaCarneVacunacionT1,ControlesCrecimDesarrolloT1,IdBeneficiarioT2,TomaT2,NoTomaT2,IdDireccionesICBFT2,DireccionICBFT2,NombreServicioT2,EdadBeneficiarioDiasT2,FechaRegistroSaludNutricionT2,FechaMedicionPerimetroBraquialT2,MedicionPerimetroBraquialT2,pesoT2,TallaT2,ZScoreTallaEdadT2,ZScorePesoEdadT2,ZScorePesoTallaT2,ZScoreIMCT2,EstadoTallaEdadT2,EstadoPesoEdadT2,EstadoPesoTallaT2,EstadoIMCT2,FlagT2,PresentaCarneVacunacionT2,ControlesCrecimDesarrolloT2,IdBeneficiarioT3,TomaT3,NoTomaT3,IdDireccionesICBFT3,DireccionICBFT3,NombreServicioT3,EdadBeneficiarioDiasT3,FechaRegistroSaludNutricionT3,FechaMedicionPerimetroBraquialT3,MedicionPerimetroBraquialT3,pesoT3,TallaT3,ZScoreTallaEdadT3,ZScorePesoEdadT3,ZScorePesoTallaT3,ZScoreIMCT3,EstadoTallaEdadT3,EstadoPesoEdadT3,EstadoPesoTallaT3,EstadoIMCT3,FlagT3,PresentaCarneVacunacionT3,ControlesCrecimDesarrolloT3,IdBeneficiarioT4,TomaT4,NoTomaT4,IdDireccionesICBFT4,DireccionICBFT4,NombreServicioT4,EdadBeneficiarioDiasT4,FechaRegistroSaludNutricionT4,FechaMedicionPerimetroBraquialT4,MedicionPerimetroBraquialT4,pesoT4,TallaT4,ZScoreTallaEdadT4,ZScorePesoEdadT4,ZScorePesoTallaT4,ZScoreIMCT4,EstadoTallaEdadT4,EstadoPesoEdadT4,EstadoPesoTallaT4,EstadoIMCT4,FlagT4,PresentaCarneVacunacionT4,ControlesCrecimDesarrolloT4,IdBeneficiarioT5,TomaT5,NoTomaT5,IdDireccionesICBFT5,DireccionICBFT5,NombreServicioT5,EdadBeneficiarioDiasT5,FechaRegistroSaludNutricionT5,FechaMedicionPerimetroBraquialT5,MedicionPerimetroBraquialT5,pesoT5,TallaT5,ZScoreTallaEdadT5,ZScorePesoEdadT5,ZScorePesoTallaT5,ZScoreIMCT5,EstadoTallaEdadT5,EstadoPesoEdadT5,EstadoPesoTallaT5,EstadoIMCT5,FlagT5,PresentaCarneVacunacionT5,ControlesCrecimDesarrolloT5,IdBeneficiarioT6,TomaT6,NoTomaT6,IdDireccionesICBFT6,DireccionICBFT6,NombreServicioT6,EdadBeneficiarioDiasT6,FechaRegistroSaludNutricionT6,FechaMedicionPerimetroBraquialT6,MedicionPerimetroBraquialT6,pesoT6,TallaT6,ZScoreTallaEdadT6,ZScorePesoEdadT6,ZScorePesoTallaT6,ZScoreIMCT6,EstadoTallaEdadT6,EstadoPesoEdadT6,EstadoPesoTallaT6,EstadoIMCT6,FlagT6,PresentaCarneVacunacionT6,ControlesCrecimDesarrolloT6,IdBeneficiarioT7,TomaT7,NoTomaT7,IdDireccionesICBFT7,DireccionICBFT7,NombreServicioT7,EdadBeneficiarioDiasT7,FechaRegistroSaludNutricionT7,FechaMedicionPerimetroBraquialT7,MedicionPerimetroBraquialT7,pesoT7,TallaT7,ZScoreTallaEdadT7,ZScorePesoEdadT7,ZScorePesoTallaT7,ZScoreIMCT7,EstadoTallaEdadT7,EstadoPesoEdadT7,EstadoPesoTallaT7,EstadoIMCT7,FlagT7,PresentaCarneVacunacionT7,ControlesCrecimDesarrolloT7,IdBeneficiarioT8,TomaT8,NoTomaT8,IdDireccionesICBFT8,DireccionICBFT8,NombreServicioT8,EdadBeneficiarioDiasT8,FechaRegistroSaludNutricionT8,FechaMedicionPerimetroBraquialT8,MedicionPerimetroBraquialT8,pesoT8,TallaT8,ZScoreTallaEdadT8,ZScorePesoEdadT8,ZScorePesoTallaT8,ZScoreIMCT8,EstadoTallaEdadT8,EstadoPesoEdadT8,EstadoPesoTallaT8,EstadoIMCT8,FlagT8,PresentaCarn

In [0]:
print(final(dfsabana,dfmodelo,pathsabana, pathpredic,path_grafica).count())

*** Borrando Directorio *** 
 *** Guardado en el DataLake *** 
 *** Borrando Directorio *** 
 *** Guardado en el DataLake *** 
 *** Borrando Directorio *** 
 *** Guardado en el DataLake *** 
52819